# Mutual Information Truths

## Normal mutual information

In [2]:
import numpy as np

In [1]:
from scipy.stats import norm
from scipy.stats import multivariate_normal
import scipy.integrate as integrate
import math
from sklearn import mixture

def estimate_p(y):
    return y.count(1)/len(y)

def estimate_x_params(X):
    model = mixture.GaussianMixture(n_components = 2, covariance_type = "full")
    model.fit(X)
    return model.means_, model.covariances_
    
estimate_x_params(X)

def normal_entropy_f(t, mean, var):
    return -norm.pdf(t, mean, var)*np.log(norm.pdf(t, mean, var))

def two_mixture_normals_entropy_f(t, mean_1, mean_2, var_1, var_2):
    return -.5*norm.pdf(t, mean_1, var_1)*np.log(.5*norm.pdf(t, mean_1, var_1) + .5*norm.pdf(t, mean_2, var_2)) - .5*norm.pdf(t, mean_1, var_1)*np.log(.5*norm.pdf(t, mean_1, var_1) + .5*norm.pdf(t, mean_2, var_2))

def normal_entropy(var):
    return .5*np.log(2*math.pi*math.e*var)

#NOTE: this doesn't work for mean = 0
def plugin_estimate_cat_1D(X, y):
    y_param = estimate_p(y)
    x_params = estimate_x_params(X)
    h_y = -y_param*np.log(y_param) - (1 - y_param)*np.log(1 - y_param)
    #h_x_cond_y = integrate.quad(normal_entropy_f, -20, 20, args = (x_params[0][0], x_params[1][0].item()))[0]*.5 + \
    #integrate.quad(normal_entropy_f, -20, 20, args = (x_params[0][1], x_params[1][1].item()))[0]*.5
    h_x_cond_y = normal_entropy(x_params[1][0])*.5 + normal_entropy(x_params[1][1])*.5
    h_x = integrate.quad(two_mixture_normals_entropy_f, -20, 20, args = (x_params[0][0], x_params[0][1], x_params[1][0].item(), x_params[1][1].item()))[0]
    cond_entropy =  h_y - h_x + h_x_cond_y
    if cond_entropy < 0:
        return np.array((0))
    return cond_entropy

def truth_1d(mean):
    h_y = -.5*np.log(.5) - .5*np.log(.5)
    h_x_cond_y = normal_entropy(1)*.5 + normal_entropy(1)*.5
    h_x = integrate.quad(two_mixture_normals_entropy_f, -20, 20, args = (mean, -mean, 1, 1))[0]
    cond_entropy =  h_y - h_x + h_x_cond_y
    if cond_entropy < 0:
        return np.array((0))
    return cond_entropy


def mutual_information_truth():
    entropy = -.5*np.log(.5) - .5*np.log(.5)
    means = [i*-.2 for i in range(0, 26)]
    means.reverse()
    means.extend([i*.2 for i in range(1, 26)])
    truths = []
    for elem in means:
        truths.append(entropy - truth_1d(elem))
    #return means, cef_all, plugin_all, truth
    return truths

NameError: name 'X' is not defined

## Different variances

In [ ]:
from scipy.stats import norm
from scipy.stats import multivariate_normal
import scipy.integrate as integrate
import math
from sklearn import mixture

def estimate_p(y):
    return y.count(1)/len(y)

def estimate_x_params(X):
    model = mixture.GaussianMixture(n_components = 2, covariance_type = "full")
    model.fit(X)
    return model.means_, model.covariances_
    
def normal_entropy_f(t, mean, var):
    return -norm.pdf(t, mean, var)*np.log(norm.pdf(t, mean, var))

def two_mixture_normals_entropy_f(t, mean_1, mean_2, var_1, var_2):
    return -.5*norm.pdf(t, mean_1, var_1)*np.log(.5*norm.pdf(t, mean_1, var_1) + .5*norm.pdf(t, mean_2, var_2)) - .5*norm.pdf(t, mean_1, var_1)*np.log(.5*norm.pdf(t, mean_1, var_1) + .5*norm.pdf(t, mean_2, var_2))

def normal_entropy(var):
    return .5*np.log(2*math.pi*math.e*var)

#NOTE: this doesn't work for mean = 0
def plugin_estimate_cat_1D(X, y):
    y_param = estimate_p(y)
    x_params = estimate_x_params(X)
    h_y = -y_param*np.log(y_param) - (1 - y_param)*np.log(1 - y_param)
    #h_x_cond_y = integrate.quad(normal_entropy_f, -20, 20, args = (x_params[0][0], x_params[1][0].item()))[0]*.5 + \
    #integrate.quad(normal_entropy_f, -20, 20, args = (x_params[0][1], x_params[1][1].item()))[0]*.5
    h_x_cond_y = normal_entropy(x_params[1][0])*.5 + normal_entropy(x_params[1][1])*.5
    h_x = integrate.quad(two_mixture_normals_entropy_f, -20, 20, args = (x_params[0][0], x_params[0][1], x_params[1][0].item(), x_params[1][1].item()))[0]
    cond_entropy =  h_y - h_x + h_x_cond_y
    if cond_entropy < 0:
        return np.array((0))
    return cond_entropy

def truth_1d(mean):
    h_y = -.5*np.log(.5) - .5*np.log(.5)
    h_x_cond_y = normal_entropy(1)*.5 + normal_entropy(3)*.5
    h_x = integrate.quad(two_mixture_normals_entropy_f, -20, 20, args = (mean, -mean, 1, math.sqrt(3)))[0]
    cond_entropy =  h_y - h_x + h_x_cond_y
    if cond_entropy < 0:
        return np.array((0))
    return cond_entropy


def mutual_information_truth():
    entropy = -.5*np.log(.5) - .5*np.log(.5)
    means=[i*.2 for i in range(0, 26)]
    truths = []
    for elem in means:
        mi = entropy - truth_1d(elem)
        if mi < 0:
            truths.append(0)
        else:
            truths.append((entropy - truth_1d(elem))/entropy)
    #return means, cef_all, plugin_all, truth
    return truths

In [28]:
truths_diff_covar = mutual_information_truth()

In [29]:
print(truths_diff_covar)

[0, 0, 0, 0, 0, 0.00139387239137524, 0.0923354486255463, 0.17983487386233699, 0.26014481159701247, 0.3311346125243729, 0.39191714540163064, 0.4424935478389682, 0.4834711645708754, 0.5158388832981347, 0.5407855870838623, 0.5595579047838196, 0.5733562865659112, 0.5832668885594593, 0.5902244414373682, 0.5949996896546803, 0.5982044676085582, 0.6003078952265687, 0.6016582208033919, 0.602506180664179, 0.6030271057430278, 0.6033401961717422]


## Different ps

In [19]:
from scipy.stats import norm
from scipy.stats import multivariate_normal
import scipy.integrate as integrate
import math
from sklearn import mixture

def estimate_p(y):
    return y.count(1)/len(y)

def estimate_x_params(X):
    model = mixture.GaussianMixture(n_components = 2, covariance_type = "full")
    model.fit(X)
    return model.means_, model.covariances_
    

def normal_entropy_f(t, mean, var):
    return -norm.pdf(t, mean, var)*np.log(norm.pdf(t, mean, var))

def two_mixture_normals_entropy_f(t, mean_1, mean_2, var_1, var_2, prob):
    return -prob*norm.pdf(t, mean_1, var_1)*np.log(prob*norm.pdf(t, mean_1, var_1) + (prob)*norm.pdf(t, mean_2, var_2)) - (1-prob)*norm.pdf(t, mean_1, var_1)*np.log((1-prob)*norm.pdf(t, mean_1, var_1) + (1-prob)*norm.pdf(t, mean_2, var_2))

def normal_entropy(var):
    return .5*np.log(2*math.pi*math.e*var)

#NOTE: this doesn't work for mean = 0
def plugin_estimate_cat_1D(X, y):
    y_param = estimate_p(y)
    x_params = estimate_x_params(X)
    h_y = -y_param*np.log(y_param) - (1 - y_param)*np.log(1 - y_param)
    #h_x_cond_y = integrate.quad(normal_entropy_f, -20, 20, args = (x_params[0][0], x_params[1][0].item()))[0]*.5 + \
    #integrate.quad(normal_entropy_f, -20, 20, args = (x_params[0][1], x_params[1][1].item()))[0]*.5
    h_x_cond_y = normal_entropy(x_params[1][0])*.5 + normal_entropy(x_params[1][1])*.5
    h_x = integrate.quad(two_mixture_normals_entropy_f, -20, 20, args = (x_params[0][0], x_params[0][1], x_params[1][0].item(), x_params[1][1].item()))[0]
    cond_entropy =  h_y - h_x + h_x_cond_y
    if cond_entropy < 0:
        return np.array((0))
    return cond_entropy

def truth_1d(prob):
    h_y = -prob*np.log(prob) - (1-prob)*np.log(1-prob)
    h_x_cond_y = normal_entropy(1)*prob + normal_entropy(1)*(1-prob)
    h_x = integrate.quad(two_mixture_normals_entropy_f, -20, 20, args = (1, -1, 1, 1, prob))[0]
    cond_entropy =  h_y - h_x + h_x_cond_y
    if cond_entropy < 0:
        return np.array((0))
    return cond_entropy


def mutual_information_truth():
    probs = [i*.02 for i in range(1, 26)]
    truths = []
    for elem in probs:
        entropy = -elem*np.log(elem) - (1-elem)*np.log(1 - elem)
        truths.append(entropy - truth_1d(elem))
    #return means, cef_all, plugin_all, truth
    return truths

In [20]:
truths=mutual_information_truth()

In [22]:
[elem for elem in truths]

[-0.2582772469333816,
 -0.18837221247894043,
 -0.12934883771250916,
 -0.07754698844452623,
 -0.031233386821665654,
 0.010608631059595974,
 0.048647124850824675,
 0.08335351918822936,
 0.11507724880731518,
 0.14408606332507468,
 0.17059160121826666,
 0.19476356787385918,
 0.21674055691830674,
 0.2366369572343613,
 0.25454794184177987,
 0.27055309735931277,
 0.28471911766804203,
 0.29710183458058814,
 0.30774776635099443,
 0.3166953067961429,
 0.32397563997903966,
 0.3296134400392592,
 0.3336273982452863,
 0.3360306068768478,
 0.3368308203468321]